In [1]:
import pandas as pd
import dask.dataframe as dd
import random
import math
from pathlib import Path
import dask
from distributed import Client, futures_of

In [2]:
c = Client()

In [3]:
df = pd.DataFrame([list(range(1000)), list(range(1000, 2000))]).T
df.columns=['a', 'b']
ddf = dd.from_pandas(df, npartitions=1)
ddf2 = dd.from_pandas(df*2, npartitions=1)

In [12]:
written = ddf.to_parquet('scratch3.parquet', compute=False)
written2 = ddf2.to_parquet('scratch4.parquet', compute=False)

In [18]:
written_persisted = dask.persist(written)

In [19]:
futures_of(written_persisted)

[<Future: finished, type: builtins.NoneType, key: ('metadata-to-parquet-4b536a6ede1edbe52faf81616f080028', 0)>]

In [22]:
c.gather(futures_of(written_persisted))

[None]

In [5]:
dask.compute(written, written2)

(None, None)

In [6]:
a = ddf2.persist()

# Read dataframes in parallel

In [23]:
c2 = Client()

/home/balast/.conda/envs/dask/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38645 instead
  warnings.warn(


In [31]:
futures = c.map(dd.read_parquet, ['row_partitioned_trade_files/row_partitioned_trade_file_000.parquet', 'row_partitioned_trade_file.parquet'])

In [32]:
a,b = c.gather(futures)

In [36]:
c2.compute(a)

<Future: pending, key: finalize-759647b45cef5ae67c914db7c815501d>